In [3]:
import os
import sys
import pandas as pd
import tensorflow as tf

# Add folder containing your model scripts
sys.path.append("forecastmodelstobeused")

# Import the exact model functions
from ceemdan_lstm import ceemdan_lstm_model
from emd_lstm import emd_lstm_model

# === Config ===
file_path = "sample.csv"  # Path to your CSV
output_dir = "forecast models"        # Destination for saved models
look_back = 6
data_partition = 0.7
i = [1, 2]  # Months to use (customize as needed)

# === Model-Target Mapping ===
export_plan = [
    ("load", ceemdan_lstm_model),
    ("solar", ceemdan_lstm_model),
    ("hydro", emd_lstm_model),
]

# === Prepare output folder ===
os.makedirs(output_dir, exist_ok=True)

# === Load dataset ===
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['timestamp'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# === Helper: ensure model is returned ===
def wrapper(model_func, *args, **kwargs):
    result = model_func(*args, **kwargs)
    if isinstance(result, tf.keras.Model):
        return result
    if isinstance(result, tuple):
        for item in reversed(result):
            if isinstance(item, tf.keras.Model):
                return item
    raise ValueError("Returned object is not a Keras model.")

# === Export Loop ===
for target, model_func in export_plan:
    print(f"\n📦 Exporting model for {target} using {model_func.__name__}...")

    try:
        new_data = df[['Month', 'Year', 'Date', target]].copy()
        cap = max(new_data[target])
        model = wrapper(model_func, new_data, i, look_back, data_partition, cap, target, return_model=True)

        export_path = os.path.join(output_dir, f"{target}.h5")
        model.save(export_path)
        print(f"✅ Saved to {export_path}")

    except Exception as e:
        print(f"❌ Failed: {target} — {e}")



📦 Exporting model for load using ceemdan_lstm_model...


c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` arg

MAPE: 1.6535150668772616
RMSE: 109.85302599781912
MAE: 80.37963163418844
✅ Saved to forecast models\load.h5

📦 Exporting model for solar using ceemdan_lstm_model...


c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` arg

MAPE: 0.13368404400223194
RMSE: 2.4488413327268286
MAE: 1.336840440022319
✅ Saved to forecast models\solar.h5

📦 Exporting model for hydro using emd_lstm_model...


c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\gepap\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` arg

MAPE: 2.1683347973057994
RMSE: 30.65130420732733
MAE: 21.68334797305799
✅ Saved to forecast models\hydro.h5
